# To-do:
* Make it user interactive

In [ ]:
# set the market <'snp'> | <'nse'>
market = 'nse'

In [ ]:
# imports
from z_helper import *

util.startLoop()

ib = get_connected(market, 'live')

# from json
a = assign_var(market) + assign_var('common')
for v in a:
    exec(v)
    
jup_disp_adjust() # adjust jupyter's display

with open(logpath+'target.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'target.log')

In [ ]:
# delete_all_data(market)

In [ ]:
%%time

#...Get the chains

if market == 'snp': # code for snp only - 35 mins

    # Download cboe weeklies to a dataframe
    dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"

    # read from row no 11, dropna and reset index
    df_cboe = pd.read_excel(dls, header=12, 
                            usecols=[0,2,3]).loc[11:, :]\
                            .dropna(axis=0)\
                            .reset_index(drop=True)

    # remove column names white-spaces and remap to IBKR
    df_cboe.columns = df_cboe.columns.str.replace(' ', '')

    # remove '/' for IBKR
    df_cboe.Ticker = df_cboe.Ticker.str.replace('/', ' ', regex=False)

    snp100 = list(pd.read_html('https://en.wikipedia.org/wiki/S%26P_100', 
                               header=0, match='Symbol')[0].loc[:, 'Symbol'])
    snp100 = [s.replace('.', ' ') if '.' in s else s  for s in snp100] # without dot in symbol

    # remove equities not in snp100
    df_symbols = df_cboe[~((df_cboe.ProductType == 'Equity') & ~df_cboe.Ticker.isin(snp100))]
    
    # rename Ticker to symbols
    df_symbols = df_symbols.rename({'Ticker': 'symbol'}, axis=1)
    
    # add in the lots
    df_symbols = df_symbols.assign(lot=100)

    # !!! DATA LIMITER !!! Get 8 symbols. 5 Equities and 3 ETFs
    # df_symbols = pd.concat([df_symbols[df_symbols.ProductType == 'Equity'].head(7), df_symbols.head(3)]) # !!! DATA LIMITER !!!
    
    instruments = [Stock(s, exchange, currency) for s in list(df_symbols.symbol)]
    
else: # code for NSE - 15 mins
    
    # extract from tradeplusonline
    tp = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')
    df_tp = tp[1][2:].iloc[:, :3].reset_index(drop=True)
    df_tp.columns=['symbol', 'lot', 'undPrice']
    df_tp = df_tp.apply(pd.to_numeric, errors='ignore') # convert lot and undPrice to numeric

    # convert symbols - friendly to IBKR
    df_tp = df_tp.assign(symbol=df_tp.symbol.str.slice(0,9))
    ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}
    df_tp.symbol = df_tp.symbol.replace(ntoi)
    df_symbols = df_tp
    
    # set the types for indexes as IND
    ix_symbols = ['NIFTY50', 'BANKNIFTY', 'NIFTYIT']
    
    # !!! DATA LIMITER !!! Get 4 symbols. 2 Equities and 2 Indexes
#     df_symbols = pd.concat([df_symbols[:2], df_symbols[df_symbols.symbol.isin(ix_symbols[:2])]]).reset_index(drop=True)

    # build the underlying contracts
    scrips = list(df_symbols.symbol)
    
    instruments = [Index(symbol=s, exchange=exchange) if s in ix_symbols else Stock(symbol=s, exchange=exchange) for s in scrips]

# ... Build chains df with undPrice
# qualify contracts (asyncio)
q_task = []
async def qual_coro(instruments):
    '''Coroutines with waits for qualification of instruments
    Arg: (instruments) as list of Stock(symbol, exchange, currency)
    Returns: awaits of qualifyContractsAsync(s)'''
    for s in instruments:
        q_task.append(ib.qualifyContractsAsync(s))
    return await asyncio.gather(*q_task)

uct = asyncio.run(qual_coro(instruments))

und_contracts = [b for a in uct for b in a]

id_sym = {u.conId: u.symbol for u in und_contracts}
print("\nCompleted qualifying underlyings\n")

# get tickers and undPrice (asyncio)
ut = [] # empty tickers

und_blks = [und_contracts[i: i+blk] for i in range(0, len(und_contracts), blk)]

for unds in und_blks:
    reqt = ib.reqTickersAsync(*unds)
    ut.append(ib.run(asyncio.wait_for(reqt, 20)))
und_ticks = [b for a in ut for b in a]

undPrice = {u.contract.conId: u.marketPrice() for u in und_ticks}
print("\nCompleted getting underlying prices\n")

# get the chains

ch_task = []
async def chains_coro(und_contracts):
    '''Get the chains for underlyings
    Arg: (und_contracts) as a list
    Returns: awaits of reqSecDefOptPramsAsyncs'''
    for c in und_contracts:
        ch_task.append(ib.reqSecDefOptParamsAsync(underlyingSymbol=c.symbol, futFopExchange='', 
                             underlyingConId=c.conId, underlyingSecType=c.secType))
    return await asyncio.gather(*ch_task)

ch = asyncio.run(chains_coro(und_contracts))

chs = [b for a in ch for b in a]

chains = {c.underlyingConId: c for c in chs}

sek = {b for a in [list(product([k], m.expirations, m.strikes)) for k, m in chains.items()] for b in a}

df_chains1 = pd.DataFrame(list(sek), columns=['undId', 'expiry', 'strike'])
df_chains1 = df_chains1.assign(undId=df_chains1.undId.astype('int32'))

df_chains1 = df_chains1.assign(symbol = df_chains1.undId.map(id_sym), 
                             undPrice = df_chains1.undId.map(undPrice))

if market == 'nse':  # code for NSE only
    # add expiryM to get lots from get_nse_lots()
    df_chains1 = df_chains1.assign(expiryM=pd.to_datetime(df_chains1.expiry).dt.strftime('%Y-%m'))
    # update lots of NSE equities
    lots = get_nse_lots()
    lots = lots.assign(expiryM=lots.expiryM.astype('str'))
    lots = lots[['symbol', 'expiryM', 'lot']].set_index(['symbol', 'expiryM'])

    chz = df_chains1.set_index(['symbol', 'expiryM'])

    df_chains = chz.join(lots).reset_index().drop('expiryM', 1)

    # For those with nan - forget the expiry!
    df_symlot = lots.reset_index().drop('expiryM', 1).drop_duplicates()
    symlot = dict(zip(df_symlot.symbol, df_symlot.lot))

    df_chains = df_chains.assign(lot=df_chains.symbol.map(symlot).fillna(df_chains.lot))
    
else: # For SNP
    df_chains = df_chains1.assign(lot=100)

df_chains = df_chains.sort_values(['symbol', 'expiry', 'strike'])
df_chains = df_chains[['symbol', 'undId', 'expiry', 'strike', 'undPrice', 'lot']].reset_index(drop=True)
df_chains = df_chains.assign(dte=df_chains.expiry.apply(get_dte))

df_chains.to_pickle(fspath+'chains.pkl')

print("\nCompleted getting chains\n")

#... Get the OHLCs (asyncio)

async def ohlc_coro(und_contracts):
    
    ohlc_task = []
    
    # build the tasks
    for qc in und_contracts:
        ohlc_task.append(ib.reqHistoricalDataAsync(contract=qc, endDateTime='', 
                                        durationStr='365 D', barSizeSetting='1 day',  
                                                    whatToShow='Trades', useRTH=True))
    return await asyncio.gather(*ohlc_task)

# make blocks of tasks
ohlcs = []

for unds in und_blks:
    ohlcs = ohlcs + asyncio.run(ohlc_coro(unds))

# make the ohlc dataframe
df_ohlc = pd.DataFrame()
for i, o in enumerate(ohlcs):
    df = util.df(o)
    if not o:
        print(f'{und_contracts[i].symbol} ohlc is empty')
    else:
        df_ohlc = df_ohlc.append(df.assign(symbol=und_contracts[i].symbol))
        
#... compute the standard deviations
df_ohlc = df_ohlc.assign(date=pd.to_datetime(df_ohlc.date, format='%Y-%m-%d'))

grp1 = df_ohlc.groupby('symbol')
grp2 = grp1.apply(lambda df: df.sort_values('date', ascending=False)).reset_index(drop=True)

df_ohlcsd = grp2.groupby('symbol').apply(lambda df: df.assign(stDev=df.close.expanding(1).std(ddof=0))).reset_index(drop=True)

df_ohlcsd.to_pickle(fspath+'ohlcs.pkl')

print("\nCompleted getting ohlcs\n")

# ... Size the chains

# replace dte with 1 for dte <= 0
df_chains.loc[df_chains.dte <=0,  'dte'] = 1
df1 = df_chains[df_chains.dte <= maxdte]

# assign right
df1 = df1.assign(right=np.where(df1.strike >= df1.undPrice, 'C', 'P'))

# generate std dataframe
dfo = df_ohlcsd[['symbol', 'stDev']]  # lookup dataframe
dfo = dfo.assign(dte=dfo.groupby('symbol').cumcount()) # get the cumulative count for location as dte
dfo.set_index(['symbol', 'dte'])

dfd = df1[['symbol', 'dte']]  # data to be looked at
dfd = dfd.drop_duplicates()  # remove duplicates

df_std = dfd.set_index(['symbol', 'dte']).join(dfo.set_index(['symbol', 'dte']))

# join to get std in chains
df2 = df1.set_index(['symbol', 'dte']).join(df_std).reset_index()

# remove the calls and puts near strike price
c_mask = (df2.right == 'C') & (df2.strike > df2.undPrice + callstdmult*df2.stDev)
p_mask = (df2.right == 'P') & (df2.strike < df2.undPrice - putstdmult*df2.stDev)
df3 = df2[c_mask | p_mask].reset_index(drop=True)

# make (df and dte) tuple for fallrise
tup4fr = [(df_ohlc[df_ohlc.symbol == s.symbol], s.dte) 
          for s in df3[['symbol', 'dte']].drop_duplicates().itertuples()]

# get the fallrise and put it into a dataframe
fr = [fallrise(*t) for t in tup4fr]
df_fr = pd.DataFrame(fr, columns=['symbol', 'dte', 'fall', 'rise' ])

# merge with options df
df3 = pd.merge(df3, df_fr, on=['symbol', 'dte'])

# make reference strikes from fall_rise
df3 = df3.assign(strikeRef = np.where(df3.right == 'P', 
                                            df3.undPrice-df3.fall, 
                                            df3.undPrice+df3.rise))
# get lo52 and hi52
df3 = df3.set_index('symbol').join(df_ohlcsd.groupby('symbol')
                                         .close.agg(['min', 'max'])
                                         .rename(columns={'min': 'lo52', 'max': 'hi52'})).reset_index()

# ...Filter # 1: Top nband for Puts and Calls > mindte
# building SELLS for further expiries
df4 = df3[df3.dte >= mindte]

gb = df4.groupby(['right'])

if 'C' in [k for k in gb.indices]:
    df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, True, True])
    df_calls = df_calls.groupby(['symbol', 'dte']).head(nBand)
else:
    df_calls = pd.DataFrame([])

if 'P' in [k for k in gb.indices]:
    df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, True, False])
    df_puts = df_puts.groupby(['symbol', 'dte']).head(nBand)
else:
    df_puts =  pd.DataFrame([])

df5 = pd.concat([df_puts, df_calls]).reset_index(drop=True)

# ....Filter # 2: nBands with fallrise for expiries less than a week
# building SELLS for nearer expiries with fallrise

df6 = df3[df3.dte < mindte]

if df6.empty: # there are no contracts below minimum dte
    df8 = pd.DataFrame([])
else:
    # get the strikes closest to the reference strikes
    df7 = df6.groupby(['symbol', 'dte'], as_index=False) \
             .apply(lambda g: g.iloc[abs(g.strike - g.strikeRef) \
             .argsort()[:nBand]]) \
             .reset_index(drop=True)

    df8 = df7.sort_values(['symbol', 'dte', 'strike'])

# make the target master
df9 = pd.concat([df8, df5], sort=False).sort_values(['symbol', 'dte', 'strike'], 
                                                    ascending=[True, True, False]).drop_duplicates().reset_index(drop=True)

# Filter out calls that don't meet the Rise criteria in the master. These are risky
df9 = df9[~((df9.right == 'C') & (df9.strike < df9.strikeRef))].reset_index(drop=True)

print("\nCompleted sizing master for target\n")

#...Get option prices and margins

# qualify the option contracts

qo_task = []
opts = [Option(i.symbol, i.expiry, i.strike, i.right, exchange) for i in df9[['symbol', 'expiry', 'strike', 'right']].itertuples()]

async def qopt_coro(contracts):
    '''Coroutines with waits for qualification of stocks
    Arg: (contracts) as list of Stock(symbol, exchange, currency)
    Returns: awaits of qualifyContractsAsync(s)'''
    for c in contracts:
        qo_task.append(ib.qualifyContractsAsync(c))
    return await asyncio.gather(*qo_task)

qual_opts = asyncio.run(qopt_coro(opts))

print("\nCompleted qualifying option contracts\n")

# get tickers and optPrice (asyncio)

opt_ct = [ct for q in qual_opts for ct in q if q] # remove []
opt_blks = [opt_ct[i: i+blk] for i in range(0, len(opt_ct), blk)]

ot = []

for opt in opt_blks:
    reqot = ib.reqTickersAsync(*opt)
    ot.append(ib.run(asyncio.wait_for(reqot, 18)))

opt_price = {t.contract.conId: t.marketPrice() 
             for opts in ot 
             for t in opts 
             if t.marketPrice() > -1} # cleaned nans

df_opt1 = util.df(opt_ct)
df_opt2 = df_opt1[list(df_opt1)[1:6]].rename(columns={'lastTradeDateOrContractMonth': 'expiry', 'conId': 'optId'})

df_opt3 = df_opt2.assign(optPrice=df_opt2.optId.map(opt_price))
df_opt4 = df_opt3[df_opt3.optPrice >= 0]

df_opt5 = pd.merge(df_opt4, df9, on=['symbol', 'expiry', 'strike', 'right'])

print("\nCompleted getting option prices\n")

#... Get the margins

# build options and orders
mgn_opts = list(df_opt5.optId.map({c.conId: c for c in opt_ct}))

if market == 'snp':
    mgn_ords = [Order(action='SELL', orderType='MKT', totalQuantity=1, whatIf=True) for r in df_opt5.lot]
else:
    mgn_ords = [Order(action='SELL', orderType='MKT', totalQuantity=r, whatIf=True) for r in df_opt5.lot]

co = list(zip(mgn_opts, mgn_ords))

task = []
async def margin_coro(co):   
    for c in co:
        task.append(ib.whatIfOrderAsync(*c))
    return await asyncio.gather(*task)

margins = asyncio.run(margin_coro(co))

# market is checked to weed out wierd commissions. NSE has commission, while SNP has maxCommission!
if market == 'nse':
    df_opt6 = df_opt5.assign(margin=[catch(lambda: float(m.initMarginChange)) for m in margins], 
                       comm=[catch(lambda: float(m.commission)) for m in margins])
else:
    df_opt6 = df_opt5.assign(margin=[catch(lambda: float(m.initMarginChange)) for m in margins], 
                       comm=[catch(lambda: float(m.maxCommission)) for m in margins])    

df_opt7 = df_opt6[df_opt6.margin < 1.7e7] # remove too high margins

df_opt8 = df_opt7.assign(PoP=[erf(i/sqrt(2.0)) for i in abs(df_opt7.strike-df_opt7.undPrice)/df_opt7.stDev],
                        RoM = abs((df_opt7.optPrice*df_opt7.lot-df_opt7.comm)/df_opt7.margin*365/df_opt7.dte))

# make reference strikes from fall_rise
df_opt9 = df_opt8.assign(strikeRef = np.where(df_opt8.right == 'P', 
                                            df_opt8.undPrice-df_opt8.fall, 
                                            df_opt8.undPrice+df_opt8.rise))

df_opt9.to_pickle(fspath+'opts.pkl')

print("\nCompleted getting margins, computing RoM and PoP and getting fallrise, hilo52\n")

#... Make the targets, Sells and Buys

#... From local pickles for action

df_chains = pd.read_pickle(fspath+'chains.pkl')
df_opt9 = pd.read_pickle(fspath+'opts.pkl')

# From Portfolio get the remaining quantitites
p = util.df(ib.portfolio()) # portfolio table

# extract option contract info from portfolio table
if p is not None:  # there are some contracts in the portfolio
    dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
    dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiry'})

    # extract the options
    dfpo = dfp[dfp.secType == 'OPT']

    # get unique symbol, lot and underlying
    df_lu = df_chains[['symbol', 'lot', 'undId', 'undPrice']].groupby('symbol').first()

    # integrate the options with lot and underlying
    dfp1 = dfpo.set_index('symbol').join(df_lu).reset_index()

    # correct the positions for nse
    if exchange == 'NSE':
        dfp1 = dfp1.assign(position=dfp1.position/dfp1.lot)

    # get the total position for options
    dfp2 = dfp1[['symbol', 'position']].groupby('symbol').sum()
    
    # Get Stock positions
    dfs1 = p[p.contract.apply(lambda x: str(x)).str.contains('Stock')]
    if not dfs1.empty:
        dfs2 = util.df(list(dfs1.contract))
        dfs3 = pd.concat([dfs2.symbol, dfs1.position.reset_index(drop=True)], axis=1)
        dfs4 = dfs3.set_index('symbol').join(df_lu)
        dfs5 = dfs4.assign(position = (dfs4.position/dfs4.lot))[['position']]
        dfp2 = dfp2.add(dfs5, fill_value=0)  # Add stock positions to option positions

    # integrate position and lots and underlyings
    dfrq1 = df_lu.join(dfp2)
    
else:
    print('There is nothing in the portfolio')
    dfrq1 = df_lu
    dfrq1['position'] = 0

# fill in the other columns
dfrq1 = dfrq1.assign(position=dfrq1.position.fillna(0)) # fillnas with zero
dfrq1 = dfrq1.assign(assVal=dfrq1.position*dfrq1.lot*dfrq1.undPrice)

assignment_limit = eval(market+'_assignment_limit')

dfrq1 = dfrq1.assign(mgnQty=-(assignment_limit/dfrq1.lot/dfrq1.undPrice))
dfrq1 = dfrq1.assign(remq=(dfrq1.position-dfrq1.mgnQty))
dfrq = dfrq1.assign(remq=dfrq1.remq.fillna(0))[['remq']]
dfrq = dfrq.assign(remq=dfrq.remq.astype('int'))

# integrate df_opt with remaining quantity

df_opt10 = df_opt9.set_index('symbol').join(dfrq).reset_index()
df_opt10 = df_opt10[df_opt10.remq > 0]  # remove options that have busted the remaining quantities

df_opt11 = df_opt10[~df_opt10.symbol.isin(blacklist)] # remove blacklists

# set the expected price and expected rom
df_opt12 = df_opt11.assign(qty=1, 
                           expPrice=np.maximum( \
                                             np.maximum(minexpRom/df_opt11.RoM*(df_opt11.optPrice+prec), minexpOptPrice), \
                                                      df_opt11.optPrice+prec))
df_opt13 = df_opt12.assign(expRom=(df_opt12.expPrice*df_opt12.lot-df_opt12.comm)/df_opt12.margin*365/df_opt12.dte)                       

# Re-adjust expPrice for expRom < minexpRom. 
# Re-calculate expRom.
# This adjustment is for optPrice = 0 and negative margin options.
mask = df_opt13.expRom < minexpRom

df_opt13[mask] = df_opt13[mask].assign(expPrice=minexpRom/df_opt13[mask].expRom*df_opt13[mask].expPrice)

df_opt13 = df_opt13.replace([np.inf, -np.inf], np.nan).dropna() # remove infinities
mask = df_opt13.expRom < minexpRom # to correct the length of the df

df_opt13 = df_opt13.assign(expPrice=[get_prec(p, prec) for p in df_opt13.expPrice])
df_opt13[mask] = df_opt13[mask].assign(expRom=(df_opt13[mask].expPrice*df_opt13[mask].lot-df_opt13[mask].comm)/df_opt13[mask].margin*365/df_opt13[mask].dte)

# symbols busting remaining quantity limit
d = {'qty': 'sumOrdQty', 'remq': 'remq'}
df_bustingrq = df_opt13.groupby('symbol').agg({'qty': 'sum', 'remq': 'mean'}).rename(columns=d)
df_bustingrq = df_bustingrq[df_bustingrq.sumOrdQty > df_bustingrq.remq].reset_index()

df_bustingrq.assign(delta=df_bustingrq.remq.sub(df_bustingrq.sumOrdQty, axis=0)).sort_values('delta')

df_opt13.to_pickle(fspath+'targets.pkl')

df_targets = pd.read_pickle(fspath+'targets.pkl').reset_index(drop=True)

print("\nCompleted creating the targets\n")

#... Get buys from existing open trades
df_buys = get_df_buys(ib, market, prec)

df_buys.to_pickle(fspath+'closing_buys.pkl')

print('\nCompleted Generating BUYs to close\n')

In [ ]:
#... Get buys from existing open trades
df_buys = get_df_buys(ib, market, prec)
df_buys

In [ ]:
# df_buys = pd.read_pickle(fspath+'closing_buys.pkl')
# df_targets = pd.read_pickle(fspath+'targets.pkl').reset_index(drop=True)

In [ ]:
from pivottablejs import pivot_ui
from IPython.core.display import HTML
pivot_ui(df_targets,outfile_path="pivottablejs.html")
HTML("pivottablejs.html")